## Chest X-Ray Pneumonia Classification – Final Model

### 🎯 Objective
The goal of this notebook is to load our winning model (MobileNetV2), confirm its performance on the test set, and save it in a final, production-ready format.

### Setup: Load Libraries and Test Data
We only need the test data generator to verify our loaded model's performance.

In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pickle

# --- Constants ---
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# --- Load and Prepare DataFrame ---
df = pd.read_csv("Data/clean_metadata.csv")
df['label_idx'] = df['label'].map({'NORMAL': 0, 'PNEUMONIA': 1})

# --- Create Test Generator ---
test_datagen = ImageDataGenerator(rescale=1./255.)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=df[df['split'] == 'test'],
    x_col='filepath',
    y_col='label_idx',
    target_size=IMG_SIZE,
    class_mode='raw',
    batch_size=BATCH_SIZE,
    shuffle=False,
    color_mode='rgb'
)

print("✅ Setup complete. Test generator is ready.")

Found 618 validated image filenames.
✅ Setup complete. Test generator is ready.


### Load the Winning Model
We'll now load the best_MobileNetV2.keras file that was saved during our comparison notebook.

In [3]:
# The path to our best performing model
model_path = "best_MobileNetV2.keras"

# Load the model
final_model = tf.keras.models.load_model(model_path)

print(f"✅ Model loaded successfully from {model_path}")
final_model.summary()

✅ Model loaded successfully from best_MobileNetV2.keras


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,261,829 (8.63 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

 Optimizer params: 2,564 (10.02 KB)

### Confirm Performance on Test Set
This is a quick sanity check to ensure we've loaded the correct model and its performance matches what we saw in the evaluation table.

In [4]:
print("--- Verifying model performance on the test set ---")

results = final_model.evaluate(test_generator)

print(f"\nTest Loss: {results[0]:.4f}")
print(f"Test Accuracy: {results[1]:.4f}")
print(f"Test AUC: {results[2]:.4f}")

print("\nPerformance is consistent with our previous evaluation. ✅")

--- Verifying model performance on the test set ---


c:\Users\Kiran\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


20/20 ━━━━━━━━━━━━━━━━━━━━ 15s 636ms/step - accuracy: 0.9094 - auc: 0.9674 - loss: 0.2255

Test Loss: 0.2255
Test Accuracy: 0.9094
Test AUC: 0.9674

Performance is consistent with our previous evaluation. ✅


### Save the Final Model
Here are the two methods for saving the model.

**Recommended Method:** Native Keras Format (.keras)

This is the safest and most reliable way. It creates a clean, portable file that contains everything needed to use the model elsewhere.

In [5]:
# Define a path for the final, production-ready model
production_model_path_keras = "pneumonia_classifier_v1.keras"

# Save the model
final_model.save(production_model_path_keras)

print(f"✅ Model saved in the recommended Keras format at: {production_model_path_keras}")

✅ Model saved in the recommended Keras format at: pneumonia_classifier_v1.keras
